In [1]:
import CoolProp 
from CoolProp.CoolProp import PropsSI
import pandas as pd
import numpy as np
import glob,sys,socket,fileinput
import os.path
import matplotlib.pyplot as plt
import scipy.signal as signal
import winsound
import tikzplotlib


#Función detectora de picos de 
#Jay: https://pythoninoffice.com/replicate-excel-vlookup-hlookup-xlookup-in-python/

def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]
    
    
#Carga de datos
D=50/1000 #Diametro de la parte cilindrica [m]
T=273.15+18
rho=PropsSI('D', 'T',T , 'P', 1e5, 'AIR') #Densidad del aire [kg/m3]
mu=PropsSI('V', 'T', T, 'P', 1e5, 'AIR') #Viscocidad del aire [Pa. s]
L=0.175*2 #Largo del cilindro en metros
A=D*L ## area proyectada

class medida:
    def __init__(self,direccion, espesor, largo):
        self.direccion = direccion
        self.espesor = espesor
        self.largo = largo
    
    def curva(self):
        a=str(str(self.direccion) + R"/e" + str(self.espesor) +R"/l"+str(self.largo))
        return a
    
#LECTURA DE DATOS
def analizadorCurva(carpeta):

    VelocityFile = "Velocidades Calibradas/Velocidades.txt"
    VelocityData = pd.read_csv(VelocityFile)
    PolinomioVelocidad=np.polyfit(VelocityData["V [V]"],VelocityData["U [m/s]"],1)

    RawDataFolder = str(carpeta)+"/"
    files = np.sort(glob.glob(RawDataFolder+'*.txt'))
    VoltajeMedicion=[]
    VelocidadMedicion=[]
    ReMedicion=[]
    for i in files:
        item=i[17:19]
        VoltajeMedicion.append(float(item))
        Vitem=PolinomioVelocidad[0]*float(item)+PolinomioVelocidad[1]
        VelocidadMedicion.append(Vitem)
        ReMedicion.append(Vitem*D*rho/mu)

    Inicial= {
        "Voltaje" : VoltajeMedicion,
        "Velocidad" : VelocidadMedicion,
        "Re" : ReMedicion,
        }

    DFInicial = pd.DataFrame(Inicial)


    DFFinal=pd.DataFrame()
    for k in files:
        #Importación y limpieza de datos de cabeza y cola
        datos = pd.read_csv(k, sep=",", header=None,skiprows=5)
        datos = datos[:-100]
        datos.columns = ["Time", "Frequency", "Drag", "Lift"]
        datos = datos.astype(float)
        datos=datos[800:2500]
        mediaLift=datos["Lift"].mean()
        stdLift=datos["Lift"].std()
        mediaDrag=(datos["Drag"].mean())
        stdDrag=(datos["Drag"].std())
        df=datos

        #Filtro para señales espurias

        for i in df.index:
            if df["Drag"][i]>(mediaDrag+200*stdDrag):
                df["Drag"][i]=np.nan
            if df["Drag"][i]<(mediaDrag-200*stdDrag):
                df["Drag"][i]=np.nan
        mediaLift=datos["Lift"].mean()
        stdLift=datos["Lift"].std()
        mediaDrag=datos["Drag"].mean()
        stdDrag=datos["Drag"].std()   

        for i in df.index:
            if df["Drag"][i]>(mediaDrag+200*stdDrag):
                df["Drag"][i]=np.nan
            if df["Drag"][i]<(mediaDrag-200*stdDrag):
                df["Drag"][i]=np.nan
        mediaLift=datos["Lift"].mean()
        stdLift=datos["Lift"].std()
        mediaDrag=datos["Drag"].mean()
        stdDrag=datos["Drag"].std()
        
        #Para evitar problemas con LS interpolo en el caso de tener una señal espuria
        df.interpolate(method ='linear', limit_direction ='backward', inplace=True)

        #Espectro de frecuencias
        tiempos=(df["Time"].values)/1000
        tiempos=tiempos-np.min(tiempos)
        fuerzas=df["Drag"].values
        fuerzas=fuerzas-np.mean(fuerzas)
        step=1/83.33
        w=np.linspace(1,2*np.pi*(1/step)*0.5,10000)
        pgram = signal.lombscargle(tiempos,fuerzas,w, normalize=True)

        dfLS=pd.DataFrame()
        dfLS["amp"]=pgram
        dfLS["frec"]=w/(2*np.pi)
        dfLS=dfLS.tail(8200)
        AmpDrag=(df["Drag"].max()-df["Drag"].min())/2
        frecpico=xlookup(dfLS["amp"].max(), dfLS["amp"], dfLS["frec"])

        #registro media, amplitud y frecuencia
        vector = pd.DataFrame(np.array([[k],[mediaLift],
                                [mediaDrag],[AmpDrag],[frecpico]]).T)
        DFFinal=pd.concat([DFFinal,vector], axis=0)

    DFFinal.rename(columns={0: 'txt', 1: 'MediaLift', 2: str("MediaDrag"),
                            3: str("AmpDrag"), 4: str("PicoDrag")}, inplace=True)
    DFFinal.reset_index(inplace=True)
    DatosCD=pd.concat([DFFinal,DFInicial],axis =1)
    
    if carpeta[0:4]=="Drag":
        RawDataFolder = "RefSoporteDrag/"
    if carpeta[0:4]=="Lift":
        RawDataFolder = "RefSoporteLift/"
    #Calculos para hacer V=f(U) y calculo de numero de Reynolds    
    files = np.sort(glob.glob(RawDataFolder+'*.txt'))
    VoltajeMedicion=[]
    VelocidadMedicion=[]
    ReMedicion=[]
    vector=[]
    for i in files:
        item=i[16:18]
        VoltajeMedicion.append(float(item))
        Vitem=PolinomioVelocidad[0]*float(item)+PolinomioVelocidad[1]
        VelocidadMedicion.append(Vitem)
        ReMedicion.append(Vitem*D*rho/mu)

    Inicial= {
        "Voltaje" : VoltajeMedicion,
        "Velocidad" : VelocidadMedicion,
        "Re" : ReMedicion,
        }
    DFInicial2 = pd.DataFrame(Inicial)
    DFFinal2 = pd.DataFrame()
    
    #Analizo el caso de referencia con el soporte
    for k in files:
        datos = pd.read_csv(k, sep=",", header=None,skiprows=5)
        datos = datos[:-100]
        datos.columns = ["Time2", "Frequency2", "Drag2", "Lift2"]
        datos = datos.astype(float)
        datos=datos[800:3000]
        mediaLift=datos["Lift2"].mean()
        stdLift=datos["Lift2"].std()
        mediaDrag=(datos["Drag2"].mean())
        stdDrag=(datos["Drag2"].std())
        df=datos

        for i in df.index:
            if df["Drag2"][i]>(mediaDrag+200*stdDrag):
                df["Drag2"][i]=np.nan
            if df["Drag2"][i]<(mediaDrag-4*stdDrag):
                df["Drag2"][i]=np.nan
        mediaLift=datos["Lift2"].mean()
        stdLift=datos["Lift2"].std()
        mediaDrag=datos["Drag2"].mean()
        stdDrag=datos["Drag2"].std()   

        for i in df.index:
            if df["Drag2"][i]>(mediaDrag+200*stdDrag):
                df["Drag2"][i]=np.nan
            if df["Drag2"][i]<(mediaDrag-200*stdDrag):
                df["Drag2"][i]=np.nan
        mediaLift=datos["Lift2"].mean()
        stdLift=datos["Lift2"].std()
        mediaDrag=datos["Drag2"].mean()
        stdDrag=datos["Drag2"].std()   


        vector = pd.DataFrame(np.array([[k],[mediaLift],[mediaDrag]]).T)
        DFFinal2=pd.concat([DFFinal2,vector], axis=0)


    DFFinal2.rename(columns={0: 'txt', 1: 'MediaLift', 2: str("MediaDrag")}, inplace=True)
    DFFinal2.reset_index(inplace=True)
    DatosCD=pd.concat([DFFinal2,DFInicial2],axis =1)
    
    
    
    ## Tomo la referencia de 0 y la borro del original

    DatosCD=pd.concat([DFFinal,DFInicial],axis =1)
    DatosCD2=pd.concat([DFFinal2,DFInicial2],axis =1)
    Referencia=DatosCD[:1]
    Drag0=Referencia["MediaDrag"][0]    DatosCD=DatosCD.drop([0]) 
    Referencia2=DatosCD2[:1]
    Drag02=Referencia2["MediaDrag"][0]    DatosCD2=DatosCD2.drop([0]) 
    DatosCD["Drag"]= DatosCD["MediaDrag"].astype(float)-float(Drag0)
    DatosCD2["Drag"]= DatosCD2["MediaDrag"].astype(float)-float(Drag02)
    
    #Elimino la componente del soporte
    DatosCD["Drag"]= DatosCD["Drag"]-DatosCD2["Drag"]    
    
    #Calculo Strouhal y la amplitud adimensional
    DatosCD["St"]=D*DatosCD["PicoDrag"].astype(float)
                    /DatosCD["Velocidad"].astype(float)
    DatosCD["AmpDrag"]=DatosCD["AmpDrag"].astype(float)*0.00980665
                    /(0.5*A*rho*DatosCD["Velocidad"]*DatosCD["Velocidad"])
    DatosCD=DatosCD.drop("txt", axis=1)
    DatosCD=DatosCD.drop("MediaDrag", axis=1)
    DatosCD=DatosCD.drop("MediaLift", axis=1)
    DatosCD=DatosCD.drop("index", axis=1)
    DatosCD[str("C"+carpeta[0])]=DatosCD["Drag"]*0.00980665
                /(0.5*A*rho*DatosCD["Velocidad"]*DatosCD["Velocidad"])    

    return DatosCD
    

In [2]:
#Carga de espesores a analizar

espesores = ["030", "075", "100", "175", "999"]
largos = ["0250", "0375", "0500", "0625", "0750"]

df_col_merged=pd.DataFrame()
#Aplico la función a toda la base de datos de medidas
for i in espesores:
    for j in largos:
        c1=analizadorCurva(medida("Drag", i, j).curva())
        c2=analizadorCurva(medida("Lift", i, j).curva())

        
        c1["Espesor"]=i
        c1["Largo"]=j
        
        c1["Lift"]=c2["Drag"]
        c1["CL"]=c2["CL"]
        c1["St Lift"] = c2["St"]
        c1["St Drag"] = c1["St"]
        c1["AmpLift"] = c2["AmpDrag"]
        c1["PicoLift"] = c2["PicoDrag"]

        df_col_merged = pd.concat([df_col_merged, c1], axis=0)

        
#Cargo los ceros

c1=analizadorCurva(medida("Drag", "000", "0000").curva())
c2=analizadorCurva(medida("Lift", "000", "0000").curva())


c1["Espesor"]="000"
c1["Largo"]="0000"
.
c1["Lift"]=c2["Drag"]
c1["CL"]=c2["CL"]
c1["St Lift"] = c2["St"]
c1["St Drag"] = c1["St"]
c1["AmpLift"] = c2["AmpDrag"]
c1["PicoLift"] = c2["PicoDrag"]

df_col_merged = pd.concat([df_col_merged, c1], axis=0)
df_col_merged = df_col_merged.applymap (float)

#Redondeo y cambio de unidades

df_col_merged=df_col_merged.round(decimals = 2)
df_col_merged["Re"]=df_col_merged["Re"].round(decimals = 0)

df_col_merged["Espesores"] = df_col_merged["Espesor"]/1000000
df_col_merged["Largo"] = df_col_merged["Largo"]/10000

#Calculo de parámetros elásticos

df_col_merged["Inercia"]= (2*0.175* df_col_merged["Espesores"]**3)/12
df_col_merged["Masa x unidad de area"] = 2*0.175* df_col_merged["Espesores"] * 1455
df_col_merged["Young"] = 0.5 * (2800 + 3100) * 1000000

lam=4.73004074
df_col_merged["Frecuencia Propia 1"] = 
    (lam**2 * (df_col_merged["Young"]*df_col_merged["Inercia"]/
               df_col_merged["Masa x unidad de area"] )**0.5)
                /(2 * np.pi * df_col_merged["Largo"]**2)
lam=7.85320462
df_col_merged["Frecuencia Propia 2"] = 
    (lam**2 * (df_col_merged["Young"]*df_col_merged["Inercia"]/
               df_col_merged["Masa x unidad de area"] )**0.5)
                /(2 * np.pi * df_col_merged["Largo"]**2)
lam=10.9956078
df_col_merged["Frecuencia Propia 3"] = 
    (lam**2 * (df_col_merged["Young"]*df_col_merged["Inercia"]/
               df_col_merged["Masa x unidad de area"] )**0.5)
                /(2 * np.pi * df_col_merged["Largo"]**2)
lam=14.1371635
df_col_merged["Frecuencia Propia 4"] = 
    (lam**2 * (df_col_merged["Young"]*df_col_merged["Inercia"]/
               df_col_merged["Masa x unidad de area"] )**0.5)
                /(2 * np.pi * df_col_merged["Largo"]**2)

#Calculo de parámetros FSI
df_col_merged["Frecuencia Adimensional 1"] = df_col_merged["Frecuencia Propia 1"]
                                                / df_col_merged["PicoLift"]
df_col_merged["Frecuencia Adimensional 2"] = df_col_merged["Frecuencia Propia 2"]
                                                / df_col_merged["PicoLift"]
df_col_merged["Frecuencia Adimensional 3"] = df_col_merged["Frecuencia Propia 3"]
                                                / df_col_merged["PicoLift"]
df_col_merged["Frecuencia Adimensional 4"] = df_col_merged["Frecuencia Propia 4"]
                                                / df_col_merged["PicoLift"]
df_col_merged["Relacion de aspecto"] = df_col_merged["Largo"]/ df_col_merged["Espesores"]
df_col_merged["Ca"] = (df_col_merged["Velocidad"]**2 * 1.2 * df_col_merged["Largo"]**3)
                        / (df_col_merged["Young"]*df_col_merged["Inercia"])